In [1]:
import pandas as pd
import pandas_profiling
import matplotlib.pyplot as plt
import numpy as np
import re
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()


# import plotly
# import pyecharts

In [2]:
# Loading data and limit to 2 million transac.
%time df = pd.read_csv('../Data/UserBehavior.csv', nrows=2000000, header=None, parse_dates=True)
# df.head()

CPU times: user 1.1 s, sys: 175 ms, total: 1.28 s
Wall time: 1.21 s


In [3]:
df.columns = ['User_Id', 'Item_Id', 'Category_Id', 'Behavior_Type', 'Time']
df.head()

,User_Id,Item_Id,Category_Id,Behavior_Type,Time
0,1,2268318,2520377,pv,1511544070
1,1,2333346,2520771,pv,1511561733
2,1,2576651,149192,pv,1511572885
3,1,3830808,4181361,pv,1511593493
4,1,4365585,2520377,pv,1511596146


In [4]:
# Trans the data type

# Trans the date dtype
df['Date'] = pd.to_datetime(df['Time'], unit='s')

# Trans the other dtype
df['Behavior_Type'] = df.Behavior_Type.astype('category')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000000 entries, 0 to 1999999
Data columns (total 6 columns):
User_Id          int64
Item_Id          int64
Category_Id      int64
Behavior_Type    category
Time             int64
Date             datetime64[ns]
dtypes: category(1), datetime64[ns](1), int64(4)
memory usage: 78.2 MB


In [18]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000000 entries, 0 to 1999999
Data columns (total 5 columns):
Date             object
User_Id          int64
Item_Id          int64
Category_Id      int64
Behavior_Type    category
dtypes: category(1), int64(3), object(1)
memory usage: 62.9+ MB


,Date,User_Id,Item_Id,Category_Id,Behavior_Type
0,2017-11-24 17:21:10,1,2268318,2520377,pv
1,2017-11-24 22:15:33,1,2333346,2520771,pv
2,2017-11-25 01:21:25,1,2576651,149192,pv
3,2017-11-25 07:04:53,1,3830808,4181361,pv
4,2017-11-25 07:49:06,1,4365585,2520377,pv


In [19]:
df['Date'] = df.Date.astype('str')
df['Date_Split'] = df.Date.str.split(' ')
df['Day'] = df['Date_Split'].str.get(0)
df['Hour'] = df['Date_Split'].str.get(1)

In [30]:
df.sort_values(['Day'], ascending=True)
df.head()
# df.Day.unique()

,Date,User_Id,Item_Id,Category_Id,Behavior_Type,Date_Split,Day,Hour
0,2017-11-24 17:21:10,1,2268318,2520377,pv,"[2017-11-24, 17:21:10]",2017-11-24,17:21:10
1,2017-11-24 22:15:33,1,2333346,2520771,pv,"[2017-11-24, 22:15:33]",2017-11-24,22:15:33
2,2017-11-25 01:21:25,1,2576651,149192,pv,"[2017-11-25, 01:21:25]",2017-11-25,01:21:25
3,2017-11-25 07:04:53,1,3830808,4181361,pv,"[2017-11-25, 07:04:53]",2017-11-25,07:04:53
4,2017-11-25 07:49:06,1,4365585,2520377,pv,"[2017-11-25, 07:49:06]",2017-11-25,07:49:06


In [31]:
drop_list = ['2017-09-11', '2018-08-28', '2017-09-16', '2018-08-28', '2017-09-16', '2017-07-03', '2015-02-06', '1970-01-01']
df[df['Day'].isin(drop_list)]

,Date,User_Id,Item_Id,Category_Id,Behavior_Type,Date_Split,Day,Hour
4170,2017-09-11 08:16:39,1000169,1328010,959452,pv,"[2017-09-11, 08:16:39]",2017-09-11,08:16:39
1046994,2018-08-28 00:30:35,130701,740615,405755,pv,"[2018-08-28, 00:30:35]",2018-08-28,00:30:35
1046995,2018-08-28 00:30:45,130701,1781671,405755,pv,"[2018-08-28, 00:30:45]",2018-08-28,00:30:45
1046996,2018-08-28 00:31:23,130701,3609645,405755,pv,"[2018-08-28, 00:31:23]",2018-08-28,00:31:23
1046997,2018-08-28 00:33:05,130701,3265412,238434,pv,"[2018-08-28, 00:33:05]",2018-08-28,00:33:05
1046998,2018-08-28 00:35:04,130701,563703,238434,pv,"[2018-08-28, 00:35:04]",2018-08-28,00:35:04
1046999,2018-08-28 00:35:22,130701,3361802,238434,pv,"[2018-08-28, 00:35:22]",2018-08-28,00:35:22
1047000,2018-08-28 00:35:42,130701,2252691,405755,pv,"[2018-08-28, 00:35:42]",2018-08-28,00:35:42
1047001,2018-08-28 00:36:54,130701,1866775,3524510,pv,"[2018-08-28, 00:36:54]",2018-08-28,00:36:54
1047002,2018-08-28 00:41:48,130701,4162858,1275696,pv,"[2018-08-28, 00:41:48]",2018-08-28,00:41:48


In [11]:
# Drop the Time column
df = df.drop(['Time'], axis=1)

In [12]:
# Sort the columns
df = df[['Date', 'User_Id', 'Item_Id', 'Category_Id', 'Behavior_Type']]
df.head()

,Date,User_Id,Item_Id,Category_Id,Behavior_Type
0,2017-11-24 17:21:10,1,2268318,2520377,pv
1,2017-11-24 22:15:33,1,2333346,2520771,pv
2,2017-11-25 01:21:25,1,2576651,149192,pv
3,2017-11-25 07:04:53,1,3830808,4181361,pv
4,2017-11-25 07:49:06,1,4365585,2520377,pv


In [ ]:
df.tail()

# 淘寶用戶行為分析

## 1. 問題挖掘
* 利用電商常用模型 AARRR 與 RFM ，來分析各階段的留客率並挖掘洞見，找出能夠改善的環節並優化。
* 利用時間規律，找出用戶的活躍傾向，並藉此安排行銷內容。
* 分析用戶對不同商品的偏好程度，進一步設計各個商品的行銷計畫。
* 找到核心的付費用戶群，並據此進行用戶畫像的繪製。

## 2. 資料概述
* 時間：2017 年 11 月 25 日至 2017 年 12 月 3 日
* 資料類別：
1. 用戶 ID
2. 物品 ID
3. 品類 ID
4. 行為類別
5. 時間戳記

## 3. 資料清理